In [2]:
import os
import torch
import mlflow
import mlflow_env
from gpt import GPTLanguageModel
from autoencoder import Autoencoder
from gpt_params import tokenizer
from torch.nn import functional as F
from gpt_params import transformer_experiment
from autoencoder_params import autoencoder_experiment, device


In [10]:
transformer_run_id = "1631cdf63904427fb5833afa9372b625"
autoencoder_run_id = "1ab8c069b70c474da0efa51bb993dae0"

gpt = GPTLanguageModel.load_from_mlflow(transformer_experiment, transformer_run_id, device)
autoencoder = Autoencoder.load_from_mlflow(autoencoder_experiment, autoencoder_run_id, device)

gpt.eval()
autoencoder.eval()

/home/test/nlp/NLP-monosemanticity/gpt.py:151: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(local_model_path, map_location=device)
/home/test/nlp/NLP-mon

Autoencoder(
  (encoder): Linear(in_features=128, out_features=2048, bias=True)
  (decoder): Linear(in_features=2048, out_features=128, bias=True)
  (relu): ReLU()
)

Forcing a neuron

In [11]:
def biased_generate(idx, max_new_tokens, neuron_idx, activation):
    if neuron_idx < 0 or neuron_idx >= autoencoder.dim_rala:
        raise ValueError(f"Invalid neuron index: {neuron_idx}")

    for _ in range(max_new_tokens):
        idx_cond = idx[:, -gpt.context_length :]
        x = gpt.embed(idx_cond)
        encoded, _ = autoencoder(x)
        encoded[:, :, neuron_idx] = activation
        x = autoencoder.decode(encoded)
        logits = gpt.unembed(x)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)
    return idx

In [29]:
idx = (
    torch.tensor(tokenizer.encode("There people born in Israel are"), dtype=torch.long)
    .unsqueeze(0)
    .to(device)
)
out = biased_generate(idx, 100, 1511, 1.5)

print(tokenizer.decode(out.squeeze().tolist()))

There people born in Israel are silent and in fact , but cited Britto Target Entertainment . Holland is one of these religions of their country called " vicious " or either ’ s values " . 


 = = Identity manifestation of the Gawixandaccioh , according to writer Paulise Leigh , blopper won as " You Really Got Your Man " ( Dr that the goal never sort of a sonic imaginary , and excitement ) . Things and Matt Patfellow : " I 'm gonna worry about ever " .
